## Declarações e imports

In [1]:
import pandas, sqlalchemy, numpy
import xml.etree.cElementTree as et

### Funções de tratamento

In [2]:
def tratamentoTelefone(telefone):
    intab = '()- '
    outtab = '    '
    transtab = str.maketrans(intab,outtab)
    return '+55'+str(telefone).translate(transtab).replace(' ','') 

def tratamentoValor(valor):
    x = lambda x: 'R$ {:.2f}'.format(float(x)).replace('.',',')
    intab = 'R$Ss'
    outtab = '    '
    transtab = str.maketrans(intab,outtab)
    return (x(valor) if str(valor).isdigit() else x(str(valor).translate(transtab).replace(',','.')))

def validaCastNumerico(valor):
    intab = 'R$Ss-'
    outtab = '     '
    transtab = str.maketrans(intab,outtab)
    valor = str(valor).replace(',','.')
    valor.translate(transtab)
    return (float(valor) if valor.isdigit() else 0)

## Não houve campo algum de data para converter para DateTime, contudo seria apenas usar 
## dataframe['campo'] = dataframe['campo'].apply(lambda x: pd.to_datetime(x))

defaultColumns = ['id','name','email','phone','value','discount', 'valor_com_desconto']

## Import do primeiro arquivo
Arquivo CSV

In [3]:
file = './dataApr-1-2019.csv'
file1 = pandas.read_csv(file, sep=';')
file1.head(5)

,client_id,username,email_client,phone_client,product_value,discount
0,1000,Cynthia Renee,nulla@sitametconsectetuer.co.uk,(051) 702-7535,75.82,66
1,1001,Kevin Zahir,Cras@justosit.org,(282) 366-4062,44.54,23
2,1002,Marvin Tucker,nec@nislsemconsequat.net,(059) 080-6768,17.12,74
3,1003,Olympia Blake,malesuada@at.edu,(928) 633-3253,6.15,33
4,1004,Wallace Ethan,sem.molestie@urnasuscipit.net,(994) 324-9907,99.76,49


## tratamento dos dados do primeiro arquivo

In [4]:
file1['valor_com_desconto'] = (file1['product_value'].apply(lambda x : float(x)) - file1['discount'].apply(lambda x : float(x)))
file1['phone_client'] = file1['phone_client'].apply(lambda x : tratamentoTelefone(x))
file1['product_value'] = file1['product_value'].apply(lambda x : tratamentoValor(x))
file1['discount'] = file1['discount'].apply(lambda x : tratamentoValor(x))
file1['valor_com_desconto'] = file1['valor_com_desconto'].apply(lambda x : tratamentoValor(x))
file1.columns = defaultColumns
file1.head(5)

,id,name,email,phone,value,discount,valor_com_desconto
0,1000,Cynthia Renee,nulla@sitametconsectetuer.co.uk,+550517027535,"R$ 75,82","R$ 66,00","R$ 9,82"
1,1001,Kevin Zahir,Cras@justosit.org,+552823664062,"R$ 44,54","R$ 23,00","R$ 21,54"
2,1002,Marvin Tucker,nec@nislsemconsequat.net,+550590806768,"R$ 17,12","R$ 74,00","R$ -56,88"
3,1003,Olympia Blake,malesuada@at.edu,+559286333253,"R$ 6,15","R$ 33,00","R$ -26,85"
4,1004,Wallace Ethan,sem.molestie@urnasuscipit.net,+559943249907,"R$ 99,76","R$ 49,00","R$ 50,76"


## Import do segundo arquivo
Arquivo do google spreadsheet

### Download do arquivo

In [5]:
## importação do google spreasheet API


### Leitura do arquivo CSV

In [6]:
file = './Teste importador - usuarios.csv'
file2 = pandas.read_csv(file, sep=',')
file2.head()

,id,nome,email,telefone,valor,desconto
0,8056,Nia Nickle,noodles@sbcglobal.net,555919295629,45.45,35
1,8094,Charisse Baltes,heine@aol.com,NaN,"45,44",12
2,8087,Shaunna Myers,tezbo@yahoo.ca,(426) 296-8278,7,45
3,8019,Del Mayorga,aukjan@icloud.com,(264) 385-2009,"45,44",-
4,8027,Kandis Marchan,lstein@mac.com,(209) 620-1483,10009,8


## tratamento dos dados do segundo arquivo

In [7]:
file2['valor_com_desconto'] = (file2['valor'].apply(lambda x : validaCastNumerico(x)) - file2['desconto'].apply(lambda x : validaCastNumerico(x)))
file2['telefone'] = file2['telefone'].mask(pandas.isnull, '')
file2['telefone'] = file2['telefone'].apply(lambda x : (tratamentoTelefone(x) if not x == numpy.nan else ''))
file2['valor'] = file2['valor'].apply(lambda x : tratamentoValor(x))
file2['desconto'] = file2['desconto'].apply(lambda x : tratamentoValor((x if str(x).isdigit() else 0)))
file2['valor_com_desconto'] = file2['valor_com_desconto'].apply(lambda x : tratamentoValor(x))
file2.columns = defaultColumns
file2.head(5)

,id,name,email,phone,value,discount,valor_com_desconto
0,8056,Nia Nickle,noodles@sbcglobal.net,+55555919295629,"R$ 45,45","R$ 35,00","R$ -35,00"
1,8094,Charisse Baltes,heine@aol.com,+55,"R$ 45,44","R$ 12,00","R$ -12,00"
2,8087,Shaunna Myers,tezbo@yahoo.ca,+554262968278,"R$ 7,00","R$ 45,00","R$ -38,00"
3,8019,Del Mayorga,aukjan@icloud.com,+552643852009,"R$ 45,44","R$ 0,00","R$ 0,00"
4,8027,Kandis Marchan,lstein@mac.com,+552096201483,"R$ 10009,00","R$ 8,00","R$ 10001,00"


## Import do terceiro arquivo
Arquivo XML

In [8]:
file = './dataApr-1-2019 2.xml'

def getvalueofnode(node):
    return node.text if node is not None else None
 
parsed_xml = et.parse(file)
dfcols = ['user_id','name','email_user','phone','buy_value']
file3 = pandas.DataFrame(columns=dfcols)

for node in parsed_xml.getroot():
    user_id = node.find('user_id')
    name = node.find('name')
    email_user = node.find('email_user')
    phone = node.find('phone')
    buy_value = node.find('buy_value')

    file3 = file3.append(
        pandas.Series([getvalueofnode(user_id),
                   getvalueofnode(name), 
                   getvalueofnode(email_user),
                   getvalueofnode(phone),
                   getvalueofnode(buy_value)], index=dfcols),
        ignore_index=True)
file3.head()

,user_id,name,email_user,phone,buy_value
0,1,Channing Hayden,cursus.et.magna@Nullamsuscipitest.ca,722 144-4062,96.55
1,2,Baker Alea,dapibus@scelerisque.co.uk,851 730-0472,93.61
2,3,Jack Kylie,dictum.Proin@necenim.net,298 821-4445,23.42
3,4,Edan Jayme,Vivamus.nibh.dolor@a.edu,123 288-9428,78.24
4,5,Hanna Kaitlin,dui@nisl.net,726 755-3600,4.17


## tratamento dos dados do terceiro arquivo

In [9]:
file3['valor_com_desconto'] = (file3['buy_value'].apply(lambda x : float(x)))
file3['phone'] = file3['phone'].apply(lambda x : tratamentoTelefone(x))
file3['buy_value'] = file3['buy_value'].apply(lambda x : tratamentoValor(x))
file3['valor_com_desconto'] = file3['valor_com_desconto'].apply(lambda x : tratamentoValor(x))
file3['discount'] = 'R$ 0.00'
file3.columns = ['id', 'name', 'email', 'phone', 'value', 'valor_com_desconto', 'discount']
file3.head(5)

,id,name,email,phone,value,valor_com_desconto,discount
0,1,Channing Hayden,cursus.et.magna@Nullamsuscipitest.ca,+557221444062,"R$ 96,55","R$ 96,55",R$ 0.00
1,2,Baker Alea,dapibus@scelerisque.co.uk,+558517300472,"R$ 93,61","R$ 93,61",R$ 0.00
2,3,Jack Kylie,dictum.Proin@necenim.net,+552988214445,"R$ 23,42","R$ 23,42",R$ 0.00
3,4,Edan Jayme,Vivamus.nibh.dolor@a.edu,+551232889428,"R$ 78,24","R$ 78,24",R$ 0.00
4,5,Hanna Kaitlin,dui@nisl.net,+557267553600,"R$ 4,17","R$ 4,17",R$ 0.00


## Juntando os e dataframes em um só

In [12]:
dataSetCompleto = pandas.concat([file1, file2, file3], sort=False)
dataSetCompleto.head(1000)

,id,name,email,phone,value,discount,valor_com_desconto
0,1000,Cynthia Renee,nulla@sitametconsectetuer.co.uk,+550517027535,"R$ 75,82","R$ 66,00","R$ 9,82"
1,1001,Kevin Zahir,Cras@justosit.org,+552823664062,"R$ 44,54","R$ 23,00","R$ 21,54"
2,1002,Marvin Tucker,nec@nislsemconsequat.net,+550590806768,"R$ 17,12","R$ 74,00","R$ -56,88"
3,1003,Olympia Blake,malesuada@at.edu,+559286333253,"R$ 6,15","R$ 33,00","R$ -26,85"
4,1004,Wallace Ethan,sem.molestie@urnasuscipit.net,+559943249907,"R$ 99,76","R$ 49,00","R$ 50,76"
5,1005,Colby Aline,ornare.sagittis@montesnascetur.net,+559688590017,"R$ 37,98","R$ 37,00","R$ 0,98"
6,1006,Carla Emerald,Suspendisse.aliquet@nonmassa.edu,+555848087730,"R$ 66,84","R$ 97,00","R$ -30,16"
7,1007,Imogene Briar,sed.turpis@nonvestibulumnec.org,+559578525089,"R$ 34,16","R$ 64,00","R$ -29,84"
8,1008,Haviva Yoshi,in@consectetueradipiscingelit.edu,+552913217760,"R$ 67,18","R$ 57,00","R$ 10,18"
9,1009,Winter Honorato,enim.Mauris.quis@elitAliquamauctor.co.uk,+550525019786,"R$ 88,89","R$ 79,00","R$ 9,89"


## Inserindo os dados em arquivo CSV

In [13]:
dataSetCompleto.to_csv(path_or_buf='./exportacao.csv',sep=';', index=False)